In [1]:
from collections import OrderedDict
def uint32(v): return (v & 0xFFFFFFFF)
def uint40(v): return (v & 0xFFFFFFFFFF)

class InstructionFormats:
    def __init__(self):
        self.formats = {
            "RRR": dict(op2=4, op1=4, r=4, s=4, t=4, op0=4),
            "RRI4": dict(imm4=4, op1=4, r=4, s=4, t=4, op0=4),
            "RRI8": dict(imm8=8, r=4, s=4, t=4, op0=4),
            "RI16": dict(imm16=16, t=4, op0=4),
            "RSR": dict(op2=4, op1=4, rs=8, t=4, op0=4),
            "CALL": dict(ofset=18, n=2, op0=4),
            "CALLX": dict(op2=4, op1=4, r=4, s=4, m=2, n=2, op0=4),
            "BRI8": dict(imm8=8, r=4, s=4, m=2, n=2, op0=4),
            "BRI12": dict(imm12=12, s=4, m=2, n=2, op0=4),
            "RRRN": dict(r=4, s=4, t=4, op0=4),
            "RI7": dict(imm7_l=4,s=4, i_imm7_h=4, op0=4),
            "RI6": dict(imm6_l=4, s=4, i_z_imm6_h=4, op0=4),
        }

    def __contains__(self, item):
        return item in self.formats
        
    def __len__(self):
        return len(self.formats)
        
    def __getitem__(self, name):
        return self.formats[name]

    def template_string(self, name, instruction):
        fmt, description, params = instruction
        fmt = self.formats[fmt]
        r = name
        sep = " "
        for p in reversed(params):
            if isinstance(p, str):
                if p.startswith("~"): continue
                if "[" in p: p = p.split("[")[0]
                r += sep + p
                sep = ","
            if isinstance(p, tuple):
                for v in reversed(p):
                    if isinstance(p, str):
                        if p.startswith("~"): continue
                        if "[" in p: p = p.split("[")[0]
                        r += sep + p
                        sep = ","
        return r + " " * (24-len(r)) + " # " + description
        
class InstructionSet(object):
    def __init__(self, name=""):
        self.name = name
        self.parent = None
        self.registers = {}
        self.instructions = {}
        self.expansions = {}
        self.endianness = "le"
        self.options = {}
        
    def get_uint_value(self, name):
        reg = self.get_value(name)
        return sum(v << (i*8) for i,v in enumerate(reg))

    def set_uint_value(self, name, value, bits):
        reg = self.set_value(name, [(value >> (i * 8)) & 0xFF for i in range(bits//8)])
        return sum(v << (i*8) for i,v in enumerate(reg))
        
    def get_value(self, name):
        r = name
        while isinstance(r, str):
            if r not in self.registers and self.parent is not None:
                return self.parent.get_value(name)
            else:
                assert r in self.registers
            r = self.registers[r]
            if isinstance(r, InstructionSetOption):
                return r.get_value(name)
        return r

    def set_value(self, name, value):
        assert isinstance(value, list)
        r = name
        while isinstance(r, str):
            if r not in self.registers and self.parent is not None:
                return self.parent.set_value(name, value)
            else:
                assert r in self.registers
            r = self.registers[r]
            if isinstance(r, InstructionSetOption):
                return r.set_value(name, value)
                
        assert len(value) <= len(r)
        for i in range(len(value)):
            r[i] = value[i]
        return r
        
    def extend(self, option):
        option.expand()
        option.parent = self
        self.options[option.name] = option
        
        for r in option.registers:
            if r in self.registers:
                print(f"WARNING. {option} overwrites register {r}")
            self.registers[r] = option #.registers[r]
        for i in option.instructions:
            if i in self.instructions:
                print(f"WARNING. {option} overwrites instruction {i}")
            self.instructions[i] = option #.instructions[i]

    def expand(self):
        for e in self.expansions:
            r = {}
            for i,v in self.instructions.items():
                if e in i:
                    for s in self.expansions[e]:
                        r[i.replace(e,s)] = v
                else:
                    r[i] = v
            self.instructions = r

    def __len__(self):
        return len(self.instructions)
        
    def __getitem__(self, name):
        return self.instructions[name]
        
class InstructionSetOption(InstructionSet): ...
class BaseInstructionSet(InstructionSet): 
    def __init__(self):
        super().__init__("base")
        self.registers = {
            "AR0": [0,0,0,0],
            "AR1": [0,0,0,0],
            "AR2": [0,0,0,0],
            "AR3": [0,0,0,0],
            "AR4": [0,0,0,0],
            "AR5": [0,0,0,0],
            "AR6": [0,0,0,0],
            "AR7": [0,0,0,0],
            "AR8": [0,0,0,0],
            "AR9": [0,0,0,0],
            "AR10": [0,0,0,0],
            "AR11": [0,0,0,0],
            "AR12": [0,0,0,0],
            "AR13": [0,0,0,0],
            "AR14": [0,0,0,0],
            "AR15": [0,0,0,0],
            "PC": [0,0,0,0],
            "SAR": [0],    
        }

        self.instructions = {
            "L8UI": ["RRI8", "8-bit unsigned load (8-bit offset)"],
            "L16SI": ["RRI8", "16-bit signed load (8-bit shifted offset)"],
            "L16UI": ["RRI8", "16-bit unsigned load (8-bit shifted offset)"],
            "L32I": ["RRI8", "32-bit load (8-bit shifted offset)"],
            "L32R": ["RI16", "32-bit load PC-relative (16-bit negative word offset)"],
            "S8I": ["RRI8", "8-bit store (8-bit offset)"],
            "S16I": ["RRI8", "16-bit store (8-bit shifted offset)"],
            "S32I": ["RRI8", "32-bit store (8-bit shifted offset)"],
            "MEMW": ["", "Order memory accesses before with memory access after"],
            "EXTW": ["", "Order all external effects before with all external effects after"],
            "CALLO": ["CALL", "Call subroutine, PC-relative"],
            "CALLX0": ["CALLX", "Call subroutine, address in register"],
            "RET": ["CALLX", "Subroutine return -- jump to return address"],
            "J": ["CALLX","Unconditional jump, PC-relative"],
            "JX": ["CALLX", "Unconditional jump, address in register"],
            "BALL": ["RRI8", "Branch if all of the masked bits are set"],
            "BNALL":["RRI8","Branch if not all of the masked bits are set"],
            "BANY": ["RRI8", "Branch if any of the masked bits are set",["label",0b1000,"as","at",0b0111]],
            "BNONE": ["RRI8", "Branch if none of the masked bits are set (All Clear)"],
            "BBC": ["RRI8", "Branch if bit clear", ["label",0b0101,"as","at",0b0111]],
            "BBCI": ["RRI8", "Branch if bit clear immediate", ["label", (0,1,1,"~bbi[4]"), "s", "bbi[0:3]", 0b0111]],
            "BBS": ["RRI8", "Branch if bit set"],
            "BBSI": ["RRI8", "Branch is bit set immediate"],
            "BEQ": ["RRI8", "Branch if equal"],
            "BEQI": ["BRI8", "Branch if equal immediate"],
            "BEQZ": ["BRI12", "Branch is equal to zero"],
            "BNE": ["RRI8", "Branch if not equal"],
            "BNEI": ["BRI8", "Branch if not equal to immediate"],
            "BNEZ": ["BRI12", "Branch if not equal to zero"],
            "BGE": ["RRI8", "Branch if greater than or equal"],
            "BGEI": ["BRI8","Branch if greater than or equal to immediate"],
            "BGEU": ["RRI8", "Branch if greater than or equal unsigned"],
            "BGEUI": ["BRI8", "Branch if greater than or equal to unsigned immediate"],
            "BGEZ": ["BRI12", "Branch to greater than or equal to zero"],
            "BLT": ["RRI8","Branch if less than"],
            "BLTI": ["BRI8", "Branch if less than immediate"],
            "BLTU": ["RRI8", "Branch if less than unsigned"],
            "BLTUI": ["BRI8", "Branch if less than unsigned immediate"],
            "BLTZ": ["RRI8", "Branch if less than zero"],
            "MOVI": ["RRI8", "Load register with 12-bit signed constant"],
            "MOVEQZ": ["RRR", "Conditional move if zero"],
            "MOVNEZ": ["RRR", "Conditional move if non-zero"],
            "MOVLTZ": ["RRR", "Conditional move if less than zero"],
            "MOVGEZ": ["RRR", "Conditional move if greater than or equal to zero"],
            "ADD": ["RRR", "Add two registers"],
            "ADDI": ["RRI8", "Add signed constant to register", ["imm8", 0b1100, "as", "at", 0b0010]],
            "ADDMI": ["RRI8", "Add signed constant shifted by 8 to register",["imm8", 0b1101, "as", "at", 0b0010]],
            "ADDX2": ["RRR", "Add register to register shifted by 1",[0b1001, 0b0000, "r","s","t",0b0000]],
            "ADDX4": ["RRR", "Add register to register shifted by 2",[0b1010, 0b0000, "r","s","t",0b0000]],
            "ADDX8": ["RRR", "Add register to register shifted by 3",[0b1011, 0b0000, "r","s","t",0b0000]],
            "SUB": ["RRR", "Subtract two registers"],
            "SUBX2": ["RRR", "Subtract register from register shifted by 1"],
            "SUBX4": ["RRR", "Subtract register from register shifted by 2"],
            "SUBX8": ["RRR", "Subtract register from register shifted by 3"],
            "NEG": ["RRR", "Negate"],
            "ABS": ["RRR", "Absolute value", [0b0110, 0b0000, "ar", 0b0001, "at", 0b0000]],
            "SALT": ["RRR", "Set AR if less than"],
            "SALTU": ["RRR", "Set AR if less than unsigned"],
            "AND": ["RRR", "Bitwise logical and", [0b0001, 0b0000, "ar","as","at", 0b0000]],
            "OR": ["RRR", "Bitwise logical or"],
            "XOR": ["RRR", "Bitwise logical xor"],
            "EXTUI": ["RRR", "Extract unsigned field immediate"],
            "SRLI": ["RRR", "Shift right logical immediate"],
            "SRAI": ["RRR", "Shift right arithmetic immediate"],
            "SLLI": ["RRR", "Shift left logical immediate"],
            "SRC": ["RRR", "Shift right combined"],
            "SLL": ["RRR", "Shift left logical"],
            "SRL": ["RRR", "Shift right logical"],
            "SRA": ["RRR", "Shift right arithmetic"],
            "SSL": ["RRR", "Set shift amount register for shift left logical"],
            "SSR": ["RRR", "Set shift amount register for shift right logical"],
            "SSAI":["RRR", "Set shift amount register immediate"],
            "SSA8B":["RRR", "Set shift amount register for big-endian byte align"],
            "SSA8L": ["RRR", "Set shift amount register for little-endian byte align"],
            "RSR": ["RSR", "Read special register"],
            "WSR": ["RSR", "Write special register"],
            "XSR": ["RSR", "Exchange special register"],
            "RUR": ["RUR", "Read user defined register"],
            "WUR": ["RUR" "Write user defined register"],
            "ISYNC": ["RRR", "Instruction fetch synchronize"],
            "RSYNC": ["RRR", "Instruction register synchronize"],
            "ESYNC": ["RRR", "Register value synchronize"],
            "DSYNC": ["RRR", "Load/store synchronize"],
            "FSYNC": ["RRR", "Fetch synchronize"],
            "NOP": ["RRR", "No operation"],
        }

        self.b4const = [-1,1,2,3,4,5,6,7,8,10,12,16,32,64,128,256]
        self.b4constu = [32768, 65536, 2,3,4,5,6,7,8,10,12,16,32,64,128,256]
        
        self.memory = [0] * (1<<23) # 8MB
        def _load(self, address, bits):
            return self.memory[address:address+(bits//8)]

        def _store(self, address, value):
            self.memory[address:address+len(value)] = value

class MAC16Option(InstructionSetOption):
    def __init__(self):
        super().__init__("mac16")
        self.registers = {
            "ACCHI": [0,0,0,0],
            "ACCLO": [0],
            "MR[0]": [0,0,0,0],
            "MR[1]": [0,0,0,0],
            "MR[2]": [0,0,0,0],
            "MR[3]": [0,0,0,0],
            "m0": "MR[0]",
            "m1": "MR[1]",
            "m2": "MR[2]",
            "m3": "MR[3]"
        }
        
        self.instructions = {
            "LDDEC": ["RRR", "Load MAC16 data register (MR) with auto decrement"],
            "LDINC": ["RRR", "Load MAC16 data register (MR) with auto increment"],
            "MUL.AA.qq": ["RRR", "Signed multiply of two address registers"],
            "MUL.AD.qq": ["RRR", "Signed multiply of an address register and a MAC16 data register"],
            "MUL.DA.qq": ["RRR", "Signed multiply of a MAC16 data register and an address register"],
            "MUL.DD.qq": ["RRR", "Signed multiply of two MAC16 data registers"],
            "MULA.AA.qq": ["RRR", "Signed multiply-accumulate of two address registers"],
            "MULA.AD.qq": ["RRR", "Signed multiply-accumulate of an address register and a MAC16 data register"],
            "MULA.DA.qq": ["RRR", "Signed multiply-accumulate of a MAC16 data register and an address register"],
            "MULA.DD.qq": ["RRR", "Signed multiply-accumulate of two MAC16 data registers"],
            "MULS.AA.qq": ["RRR", "Signed multiply/subtract of two address registers"],
            "MULS.AD.qq": ["RRR", "Signed multiply/subtract of an address register and a MAC16 data register"],
            "MULS.DA.qq": ["RRR", "Signed multiply/subtract of a MAC16 data register and an address register"],
            "MULS.DD.qq": ["RRR", "Signed multiply/subtract of two MAC16 data registers"],
            "MULA.DA.qq.LDDEC": ["RRR", "Signed multiply-accumulate of a MAC16 data register and an address register, and load a MAC16 data register with auto decrement"],
            "MULA.DA.qq.LDINC": ["RRR", "Signed multiply-accumulate of a MAC16 data register and an address register, and load a MAC16 data register with auto increment"],
            "MULA.DD.qq.LDDEC": ["RRR", "Signed multiply-accumulate of two MAC16 data registers, and load a MAC16 data register with auto decrement"],
            "MULA.DD.qq.LDINC": ["RRR", "Signed multiply-accumulate of two MAC16 data registers, and load a MAC16 data register with auto increment"],
            "UMUL.AA.qq": ["RRR", "Unsigned multiply of two address registers"],
        }

        self.expansions = {
            "qq": ["LL","HL","LH","HH"],
        }

class WindowedRegisterOption(InstructionSetOption):
    def __init__(self):
        super().__init__("windowed_register")
        self.registers = {
            "_AR0": [0,0,0,0],
            "_AR1": [0,0,0,0],
            "_AR2": [0,0,0,0],
            "_AR3": [0,0,0,0],
            "_AR4": [0,0,0,0],
            "_AR5": [0,0,0,0],
            "_AR6": [0,0,0,0],
            "_AR7": [0,0,0,0],
            "_AR8": [0,0,0,0],
            "_AR9": [0,0,0,0],
            "_AR10": [0,0,0,0],
            "_AR11": [0,0,0,0],
            "_AR12": [0,0,0,0],
            "_AR13": [0,0,0,0],
            "_AR14": [0,0,0,0],
            "_AR15": [0,0,0,0],
            "_AR16": [0,0,0,0],
            "_AR17": [0,0,0,0],
            "_AR18": [0,0,0,0],
            "_AR19": [0,0,0,0],
            "_AR20": [0,0,0,0],
            "_AR21": [0,0,0,0],
            "_AR22": [0,0,0,0],
            "_AR23": [0,0,0,0],
            "_AR24": [0,0,0,0],
            "_AR25": [0,0,0,0],
            "_AR26": [0,0,0,0],
            "_AR27": [0,0,0,0],
            "_AR28": [0,0,0,0],
            "_AR29": [0,0,0,0],
            "_AR30": [0,0,0,0],
            "_AR31": [0,0,0,0],
            "_AR32": [0,0,0,0],
            "_AR33": [0,0,0,0],
            "_AR34": [0,0,0,0],
            "_AR35": [0,0,0,0],
            "_AR36": [0,0,0,0],
            "_AR37": [0,0,0,0],
            "_AR38": [0,0,0,0],
            "_AR39": [0,0,0,0],
            "_AR40": [0,0,0,0],
            "_AR41": [0,0,0,0],
            "_AR42": [0,0,0,0],
            "_AR43": [0,0,0,0],
            "_AR44": [0,0,0,0],
            "_AR45": [0,0,0,0],
            "_AR46": [0,0,0,0],
            "_AR47": [0,0,0,0],
            "_AR48": [0,0,0,0],
            "_AR49": [0,0,0,0],
            "_AR50": [0,0,0,0],
            "_AR51": [0,0,0,0],
            "_AR52": [0,0,0,0],
            "_AR53": [0,0,0,0],
            "_AR54": [0,0,0,0],
            "_AR55": [0,0,0,0],
            "_AR56": [0,0,0,0],
            "_AR57": [0,0,0,0],
            "_AR58": [0,0,0,0],
            "_AR59": [0,0,0,0],
            "_AR60": [0,0,0,0],
            "_AR61": [0,0,0,0],
            "_AR62": [0,0,0,0],
            "_AR63": [0,0,0,0],
            "AR0": "_AR0",
            "AR1": "_AR1",
            "AR2": "_AR2",
            "AR3": "_AR3",
            "AR4": "_AR4",
            "AR5": "_AR5",
            "AR6": "_AR6",
            "AR7": "_AR7",
            "AR8": "_AR8",
            "AR9": "_AR9",
            "AR10": "_AR10",
            "AR11": "_AR11",
            "AR12": "_AR12",
            "AR13": "_AR13",
            "AR14": "_AR14",
            "AR15": "_AR15",
            "WindowBase": [0],
            "WindowStart": [0,0],
            "PS.CALLINC": [0],
            "PS.OWB": [0],
            "PS.WOE": [0],
        }

        self.instructions = {
            "MOVSP": ["RRR", "Atomic check window and move"],
            "CALL4": ["CALLX", "Call subroutine, PC-relative, hide four registers"],
            "CALL8": ["CALLX", "Call subroutine, PC-relative, hide eight registers"],
            "CALL12": ["CALLX", "Call subroutine, PC-relative, hide twelve registers"],
            "CALLX4": ["CALLX", "Call subroutine, address in register, hide four registers"],
            "CALLX8": ["CALLX", "Call subroutine, address in register, hide eight registers"],
            "CALLX12": ["CALLX", "Call subroutine, address in register, hide twelve registers"],
            "ENTRY": ["BRI12", "Subroutine entry, rotate registers, adjust stack pointer"],
            "RETW": ["CALLX", "Subroutine return, unrotate registers, jump to return address"],
            "RETW.N": ["CALLX", "Subroutine return, unrotate registers, jump to return address (16-bit encoding)"],
            "ROTW": ["RRR", "Rotate window by a constant"],
            "L32E": ["RRI4", "Load 32 bits for window exception"],
            "S32E": ["RRI4", "Store 32 bits for window exception"],
            "RFWO": ["RRR", "Return from window overflow exception"],
            "RFWU": ["RRR", "Return from window underflow exception"],
        }

    def _entry(self, imm12, s):
        assert s >= 0 and s <= 3
        
        callinc = self.get_uint_value("PS.CALLINC")
        window_base = self.get_uint_value("WindowBase")
        window_start = self.get_uint_value("WindowStart")
        window_start |= 1 << window_base
        window_base = self.set_uint_value("WindowBase", window_base + (callinc << 2), 8)
        window_start = self.set_uint_value("WindowStart", window_start, 16)
        
        sp_reg = "AR" + str(s)
        sp_val = self.get_uint_value(sp_reg)
        sp_val = self.set_uint_value(sp_reg, sp_val - (imm12 << 3), 32)
        for i in range(16):
            dst = i + window_base
            self.registers["AR" + str(i)] = ("_AR" + str(dst)) if dst < 64 else None
        self.set_uint_value( sp_reg, sp_val, 32)
        a0 = self.get_uint_value("AR0")
        a0 = self.set_uint_value("AR0", (a0 & 0x3FFFFFFF) | (callinc << 30), 32)
        return self

    def _retw(self):
        # TODO window overflow / underflow + exceptions
        pc = self.get_uint_value("PC")
        ws = self.get_uint_value("WindowStart")
        address = self.get_uint_value("AR0")
        callinc = (address >> 30) & 0x3
        address = (address & 0x3FFFFFFF) | (pc & 0xC0000000)
        window_base = self.get_uint_value("WindowBase")
        window_base = self.set_uint_value("WindowBase", window_base - (callinc << 2), 8)
        for i in range(16):
            dst = i + window_base
            self.registers["AR" + str(i)] = ("_AR" + str(dst)) if dst < 64 else None
        if ws & (1<<window_base):
            ws &= ~(1<<window_base)
            self.set_uint_value("WindowStart", ws, 16)
            self.set_uint_value("PS.CALLINC", callinc, 8)
            self.set_uint_value("PC", address, 32)
        
class MiscellaneousOperationsOption(InstructionSetOption):
    def __init__(self):
        super().__init__("miscellaneous")
        self.registers = {}
        self.instructions = {
            "CLAMPS": ["RRR", "Clamp to signed power of two range"],
            "MAX": ["RRR", "Max value signed"],
            "MAXU": ["RRR", "Max value unsigned"],
            "MIN": ["RRR", "Min value signed"],
            "MINU": ["RRR", "Min value signed"],
            "NSA": ["RRR", "Normalization shift amount signed"],
            "NSAU": ["RRR", "Normalization shift amount unsigned"],
            "SEXT": ["RRR", "Sign extend"]
        }

class FloatingPointCoprocessorOption(InstructionSetOption):
    def __init__(self, double_precision=True):
        super().__init__("floating_point_coprocessor")
        self.registers = {
            "FR0": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR1": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR2": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR3": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR4": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR5": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR6": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR7": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR8": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR9": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR10": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR11": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR12": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR13": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR14": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR15": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FCR": [0,0,0,0],
            "FSR": [0,0,0,0],
        }

        self.instructions = {
            "ABS.p": ["RRR", "Absolute value", [0b1111, "p", "fr","fs", 0b0001, 0b0000]],
            "ADD.p": ["RRR", "Add", [0b0000, "p", "fr","fs","ft", 0b0000]],
            "ADDEXP.p": ["RRR", "Add exponent", [0b1111, "p","fr","fs",0b1110, 0b0000]],
            "ADDEXPM.p": ["RRR", "Add exponent from mantissa segment", [0b1111, "p", "fr","fs",0b1111,0b0000]],
            "CEIL.p": ["RRR", "Floating-point to signed integer conversion with round to +inf"],
            "CONST.p": ["RRR", "Create floating-point constant"],
            "DIV0.p": ["RRR", "IEEE divide initial step"],
            "DIVN.p": ["RRR", "IEEE divide final step"],
            "FLOAT.p": ["RRR", "Signed integer to floating-point conversion (current rounding mode"],
            "FLOOR.p": ["RRR", "Floating-point to signed integer conversion with round to -inf"],
            "LpI": ["RRI8", "Load immediate"],
            "LpIP": ["RRI8", "Load immediate with base post-increment"],
            "LpX": ["RRR", "Load indexed"],
            "LpXP": ["RRR", "Load with base post-increment"],
            "MADD.p": ["RRR", "Multiply-add"],
            "MADDN.p": ["RRR", "Multiply-add with round mode override to round-to-nearest"],
            "MKDADJ.p": ["RRR", "Make divide adjust amounts"],
            "MKSADJ.p": ["RRR", "Make square-root adjust amounts"],
            "MOV.p": ["RRR", "Move"],
            "MOVEQZ.p": ["RRR", "Move if equal to zero"],
            "MOVF.p": ["RRR", "Move if boolean condition false"],
            "MOVGEZ.p": ["RRR", "Move if greater than or equal to zero"],
            "MOVLTZ.p": ["RRR", "Move if less than zero"],
            "MOVNEZ.p": ["RRR", "Move if not equal to zero"],
            "MOVT.p": ["RRR", "Move if boolean condition true"],
            "MSUB.p": ["RRR", "Multiply-subtract"],
            "MUL.p": ["RRR", "Multiply"],
            "NEG.p": ["RRR", "Negate"],
            "NEXP01.p": ["RRR", "Narrow exponent"],
            "OEQ.p": ["RRR", "Compare equal"],
            "OLE.p": ["RRR", "Compare less than or equal"],
            "OLT.p": ["RRR", "Compare less than"],
            "RECIP0.p": ["RRR", "Reciprocal initial step"],
            "RFR": ["RRR", "Read floating-point register (FR to AR)"],
            "ROUND.p": ["RRR", "Floating-point to signed integer conversion with round to nearest"],
            "RSQRT0.p": ["RRR", "Reciprocal square root initial step"],
            "RUR.FCR": ["RRR", "Read floating-point control register (to AR)"],
            "RUR.FSR": ["RRR", "Read floating-point status register (to AR)"],
            "SQRT0.p": ["RRR", "Square root initial step"],
            "SSI": ["RRI8", "Store immediate"],
            "SSIP": ["RRI8", "Store immidiate with base post-increment"],
            "SSX": ["RRR", "Store indexed"],
            "SSXP": ["RRR", "Store indexed with base post-increment"],
            "SUB.p": ["RRR", "Subtract"],
            "TRUNC.p": ["RRR", "Floating-point to signed integer conversion with round to 0"],
            "UEQ.p": ["RRR", "Compare unordered or equal"],
            "UFLOAT.p": ["RRR", "Unsigned integer to floating-point conversion (current rounding mode"],
            "ULE.p": ["RRR", "Compare unordered or less than or equal"],
            "ULT.p": ["RRR", "Compare unordered or less than"],
            "UN.p": ["RRR", "Compare unordered"],
            "UTRUNC.p": ["RRR", "Floating-point to unsigned integer conversion with round to 0"],
            "WFR": ["RRR", "Write floating-point register (AR to FR)"],
            "RUR.FCR": ["RRR", "Write floating-point control register"],
            "RUR.FSR": ["RRR", "Write floating-point status register"],
            # double precision only
            "CVTD.S": ["RRR", "Convert single-precision to double-precision"],
            "CVTS.D": ["RRR", "Convert double-precision to single-precision"],
            "RFRD": ["RRR", "Read floating-point register upper (FR to AR)"],
        }

        self.expansions = {
            "p": { "S": 0b1010, "D":0b1111 } 
        }

class ESPS3ExtensionOption(InstructionSetOption):
    def __init__(self):
        super().__init__("esp32s3_extension")
        self.registers = {
            "Q0": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q1": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q2": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q3": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q4": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q5": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q6": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q7": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "q0": "Q0",
            "q1": "Q1",
            "q2": "Q2",
            "q3": "Q3",
            "q4": "Q4",
            "q5": "Q5",
            "q6": "Q6",
            "q7": "Q7",
            "SAR_BYTE": [0],
            "ACCX": [0,0,0,0,0],
            "QACC_H": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "QACC_L": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "FFT_BIT_WIDTH": [0],
            "UA_STATE": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        }

        self.instructions = {
            "LD.QR": ["","Load 16-byte data to QR"],
            "EE.VLD.128.p": ["", "Read the 16-byte data, then add a value to the access address."],
            "EE.VLD.hl.64.p": ["", "Read the 8-byte data, then add a value to the access address"],
            "EE.VLDBC.n32.q": [],
            "EE.VLDHBC.16.INCP": [],
            "EE.LDF.m.p": [],
            "EE.LD.128.USAR.p": [],
            "EE.LDQA.usn16.128.p": [],
            "EE.LD.QACC_hl.z.IP": [],
            "EE.LD.ACCX.IP": [],
            "EE.LD.UA_STATE.IP": [],
            "EE.LDXQ.32": [],
            "ST.QR": [],
            "EE.VST.128.p": [],
            "EE.VST.hl.64.p": [],
            "EE.STF.m.p": [],
            "EE.ST.QACC_hl.z.IP": [],
            "EE.ST.ACCX.IP": [],
            "EE.ST.UA_STATE.IP": [],
            "EE.STXQ.32": [],
            "MV.QR": [],
            "EE.MOVI.32.A": [],
            "EE.MOVI.32.Q": [],
            "EE.VZIP.n32": [],
            "EE.VUNZIP.n32": [],
            "EE.ZERO.Q": [],
            "EE.ZERO.QACC": [],
            "EE.ZERO.ACCX": [],
            "EE.MOV.usn16.QACC": [],
            "EE.VADDS.Sn32.w1": [],
            "EE.VSUBS.Sn32.w1": [],
            "EE.VMUL.usn16.w1": [],
            "EE.CMUL.S16.w1": [],
            "EE.VMULAS.usn16.ACCX.w2": [],
            "EE.VMULAS.usn16.QACC.w3": [],
            "EE.VMULAS.usn16.ACCX.w2.QUP": [],
            "EE.VMULAS.usn16.QACC.w3.QUP": [],
            "EE.SMULAS.Sn16.QACC.w4": [],
            "EE.SRCMB.Sn16.QACC": [],
            "EE.SRS.ACCX": [],
            "EE.VRELU.Sn8": [],
            "EE.VPRELU.Sn16": [],
            "EE.VMAX.Sn32.w1": [],
            "EE.VMIN.Sn32.w1": [],
            "EE.VCMP.ee.Sn32": [],
            "EE.ORQ": [],
            "EE.XORQ": [],
            "EE.ANDQ": [],
            "EE.NOTQ": [],
            "EE.SRC.Q": [],
            "EE.SRC.Q.QUP": [],
            "EE.SRC.Q.LD.p": [],
            "EE.SLCI.2Q": [],
            "EE.SLCXXP.2Q": [],
            "EE.SRCI.2Q": [],
            "EE.SRCXXP.2Q": [],
            "EE.SRCQ.128.ST.INCP": [],
            "EE.VSR.32": [],
            "EE.VSL.32": [],
            "EE.FFT.R2BF.S16": [],
            "EE.FFT.R2BF.S16.ST.INCP": [],
            "EE.FFT.CMUL.S16.w5": [],
            "EE.BITREV": [],
            "EE.FFT.AMS.S16.w6": [],
            "EE.FFT.VST.R32.DECP": [],
            "EE.WR_MASK_GPIO_OUT": [],
            "EE.SET_BIT_GPIO_OUT": [],
            "EE.CLR_BIT_GPIO_OUT": [],
            "EE.GET_GPIO_IN": [],
        }

        self.expansions = {
            "p": { "IP": 1, "XP":2 },
            "q": { "": 0, "IP": 1, "XP": 2 },
            "n16": { "8": 3, "16": 4 },
            "n32": { "8": 3, "16": 4, "32": 5 },
            "m": { "64": 0, "128": 1 },
            "us": { "U": 0x0000, "S": 0x0001 },
            "hl": { "H": 0b0000, "L": 0b0001 },
            "z": {"H.32": 0, "L.128":1 },
            "w1": { "": 0, "LD.INCP": 1, "ST.INCP": 0 },
            "w2": { "": 0, "LD.IP": 1, "LD.XP": 2},
            "w3": { "": 0, "LD.IP": 1, "LD.XP": 2, "LDBC.INCP":3},
            "w4": { "": 0, "LD.INCP": 1},
            "w5": { "LD.XP": 0, "ST.XP": 1 },
            "w6": { "LD.INCP.UAUP":0, "LD.INCP": 1, "LD.R32.DECP": 2, "ST.INCP": 3 },
            "ee": { "EQ": 0, "LT": 1, "GT": 2 },
        }
        
base = BaseInstructionSet()
base.expand()
base.extend(MAC16Option())
base.extend(WindowedRegisterOption())
InstructionFormat = InstructionFormats()

instruction = base["ADDMI"]
print(InstructionFormat.template_string("ADDMI", instruction))

base.set_uint_value("PC", 128, 32)
base.set_uint_value("AR3", 1024, 32)
base.set_uint_value("AR12", 13, 32)
base.set_uint_value("AR0", 132, 32)

print("AR0", base.get_uint_value("AR0"))
print("AR3", base.get_uint_value("AR3"))

base.set_uint_value("PS.CALLINC", 2, 8)
base.options["windowed_register"]._entry(32,3)
base.set_uint_value("AR2", 21, 32)

print("AR0 = 21?", base.get_uint_value("AR0"))
print("AR3 = 768?", base.get_uint_value("AR3"))
print("AR4 = 13?", base.get_uint_value("AR4"))

base.options["windowed_register"]._retw()
print("AR0 = 132?", base.get_uint_value("AR0"))
print("AR3 = 1024?", base.get_uint_value("AR3"))
print("AR10 = 21?", base.get_uint_value("AR10"))
print("AR12 = 13?", base.get_uint_value("AR12"))


WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR0
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR1
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR2
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR3
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR4
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR5
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR6
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR7
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR8
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites register AR9
WARNING. <__main__.WindowedRegisterOption object at 0x76081aab4dc0> overwrites r